# 1 - 不是一直要用锤子

面对页面解析难题（Gordian Knot）的时候，不假思索地直接写几行语句来抽取信息是非常直接的做法。但是，像这样鲁莽放纵地使用技术，只会让程序变得难以调试或脆弱不堪，甚至二者兼具。

- 寻找“打印此页”的链接，或者看看网站有没有HTML样式更友好的移动版（把自己的请求头设置成处于移动设备的状态，然后接收网站移动版）。
- 寻找隐藏在JavaScript文件里的信息。要实现这一点，你可能需要查看网页加载的JavaScript文件。我曾经要把一个网站上的街道地址（以经度和纬度呈现的）整理成格式整洁的数组时，查看过内嵌谷歌地图的JavaScript文件，里面有每个地址的标记点。
- 虽然网页标题经常会用到，但是这个信息也许可以从网页的URL链接里获取。
- 如果你要找的信息只存在于一个网站上，别处没有，那你确实是运气不佳。如果不只限于这个网站，那么你可以找找其他数据源。有没有其他网站也显示了同样的数据？网站上显示的数据是不是从其他网站上抓取后攒出来的？

# 2 - 再端一碗BeautifulSoup

基本上，你见过的每个网站都会有层叠样式表（Cascading Style Sheet，CSS）。虽然你可能会认为，专门为了让浏览器和人类可以理解网站内容而设计一个展现样式的层，是一件愚蠢的事，但是CSS的发明却是网络爬虫的福音。CSS可以让HTML元素呈现出差异化，使那些具有完全相同修饰的元素呈现出不同的样式。比如，有一些标签看起来是这样：

In [ ]:
<span class="green"></span>

而另一些标签看起来是这样的：

In [ ]:
<span class="red"></span>

网络爬虫可以通过class属性的值，轻松地区分出两种不同的标签。例如，它们可以用BeautifulSoup抓取网页上所有的红色文字，而绿色文字一个都不抓。因为CSS通过属性准确地呈现网站的样式，所以你大可放心，大多数新式网站上的class和id属性资源都非常丰富。

下面让我们创建一个网络爬虫来抓取http://www.pythonscraping.com/pages/warandpeace.html 这个网页。在这个页面里，小说人物的对话内容都是红色的，人物名称都是绿色的。

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/warandpeace.html")
bs0bj = BeautifulSoup(html)
namelist = bs0bj.findAll("span",{"class":"green"})
for name in namelist:
    print(name.get_text())

### 什么时候使用get_text()与什么时候应该保留标签？
- get_text()会把你正在处理的HTML文档中所有的标签都清除，然后返回一个只包含文字的字符串。假如你正在处理一个包含许多超链接、段落和标签的大段源代码，那么.get_text()会把这些超链接、段落和标签都清除掉，只剩下一串不带标签的文字。
- 用BeautifulSoup对象查找你想要的信息，比直接在HTML文本里查找信息要简单得多。通常在你准备打印、存储和操作数据时，应该最后才使用.get_text()。一般情况下，你应该尽可能地保留HTML文档的标签结构。

# 3 - BeautifulSoup的find()和findAll()

BeautifulSoup里的find()和findAll()可能是你最常用的两个函数。借助它们，你可以通过标签的不同属性轻松地过滤HTML页面，查找需要的标签组或单个标签。

find(tag, attributes, recursive, text, limit, keywords)

findAll(tag, attributes, recursive, text, keywords)

- 标签参数tag前面已经介绍过——你可以传一个标签的名称或多个标签名称组成的Python列表做标签参数。例如，下面的代码将返回一个包含HTML文档中所有标题标签的列表：

In [ ]:
tag_list = bs0bj.findAll({"h1","h2","h3","h4","h5","h6"})
print(tag_list)

- 属性参数attributes是用一个Python字典封装一个标签的若干属性和对应的属性值。例如，下面这个函数会返回HTML文档里红色与绿色两种颜色的span标签：

In [ ]:
tag_list = bs0bj.findAll("span", {"class":{"green","red"}})
print(tag_list)

- 递归参数recursive是一个布尔变量。你想抓取HTML文档标签结构里多少层的信息？如果recursive设置为True，findAll就会根据你的要求去查找标签参数的所有子标签，以及子标签的子标签。如果recursive设置为False，findAll就只查找文档的一级标签。findAll默认是支持递归查找的（recursive默认值是True）；一般情况下这个参数不需要设置，除非你真正了解自己需要哪些信息，而且抓取速度非常重要，那时你可以设置递归参数。

- 文本参数text有点不同，它是用标签的文本内容去匹配，而不是用标签的属性。假如我们想查找前面网页中包含“the prince”内容的标签数量，我们可以把之前的findAll方法换成下面的代码：

In [ ]:
content_list = bs0bj.findAll(text="the prince")
print(len(content_list))

- 范围限制参数limit，显然只用于findAll方法。find其实等价于findAll的limit等于1时的情形。如果你只对网页中获取的前x项结果感兴趣，就可以设置它。但是要注意，这个参数设置之后，获得的前几项结果是按照网页上的顺序排序的，未必是你想要的那前几项。

- 还有一个关键词参数keyword，可以让你选择那些具有指定属性的标签。例如：

In [ ]:
# 选择包含属性id='text'的标签
allText = bs0bj.findAll(id='text')
print(allText[0].get_text())

### 关键词参数的注意事项

虽然关键词参数keyword在一些场景中很有用，但是，它是BeautifulSoup在技术上做的一个冗余功能。任何用关键词参数能够完成的任务，同样可以用本章后面将介绍的技术解决。例如，下面两行代码是完全一样的：

In [ ]:
bs0bj.findAll(id="text")
bs0bj.findAll("",{"id":"text"})

另外，用keyword偶尔会出现问题，尤其是在用class属性查找标签的时候，因为class是Python中受保护的关键字。也就是说，class是Python语言的保留字，在Python程序里是不能当作变量或参数名使用的（和前面介绍的BeautifulSoup.findAll()里的keyword无关）2。假如你运行下面的代码，Python就会因为你误用class保留字而产生一个语法错误：

In [ ]:
bs0bj.findAll(class="green")

不过，你可以用BeautifulSoup提供的有点儿臃肿的方案，在class后面增加一个下划线：

In [ ]:
bs0bj.findAll(class_ = "green")

另外，你也可以用属性参数把class用引号包起来：

In [ ]:
bs0bj.findAll("", {"class":"green"})

# 4 - 其他BeautifulSoup对象

- NavigableString对象
    - 用来表示标签里的文字，不是标签（有些函数可以操作和生成NavigableString对象，而不是标签对象）。
- Comment对象
    - 用来查找HTML文档的注释标签，\<!--像这样--> 

# 5 - 导航树

findAll函数通过标签的名称和属性来查找标签。但是如果你需要通过标签在文档中的位置来查找标签，该怎么办？这就是导航树（Navigating Trees）的作用。在第1章里，我们看过用单一方向进行BeautifulSoup标签树的导航：

bsObj.tag.subTag.anotherSubTag

现在我们用虚拟的在线购物网站http://www.pythonscraping.com/pages/page3.html 作为要抓取的示例网页，演示HTML导航树的纵向和横向导航。

## 5.1 - 处理子标签和其他后代标签

在计算机科学和一些数学领域中，你经常会听到“虐子”事件（比喻对一些子事件的处理方式）：移动它们，储存它们，删除它们，甚至杀死它们。值得庆幸的是，在BeautifulSoup里，子标签的处理方式没那么残忍。

和许多其他库一样，在BeautifulSoup库里，孩子（child）和后代（descendant）有显著的不同：和人类的家谱一样，子标签就是一个父标签的下一级，而后代标签是指一个父标签下面所有级别的标签。例如，tr标签是tabel标签的子标签，而tr、th、td、img和span标签都是tabel标签的后代标签（我们的示例页面中就是如此）。所有的子标签都是后代标签，但不是所有的后代标签都是子标签。

一般情况下，BeautifulSoup函数总是处理当前标签的后代标签。例如，bs0bj.body.h1选择了body标签后代里的第一个h1标签，不会去找body外面的标签。

类似地，bs0bj.div.findAll("img")会找出文档中第一个div标签，然后获取这个div后代里所有的img标签列表。

如果只想找出子标签，可以使用.children标签。

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/page3.html")
bs0bj = BeautifulSoup(html)
# 只打印子标签
for child in bs0bj.find("table", {"id":"giftList"}).children:
    print(child)

这段代码会打印giftList表格中所有产品的数据行。如果你用descendants()函数而不是children()函数，那么就会有二十几个标签打印出来，包括img标签、span标签，以及每个td标签。掌握子标签与后代标签的差别十分重要！

In [ ]:
# 打印所有后代标签
for descendant in bs0bj.find("table", {"id":"giftList"}).descendants:
    print(descendant)

## 5.2 - 处理兄弟标签

BeautifulSoup的next_siblings()函数可以让收集表格数据成为简单的事情，尤其是处理带标题行的表格：

In [ ]:
for tag in bs0bj.find("table", {"id":"giftList"}).tr.next_siblings:
    print(tag)

这段代码会打印产品列表里的所有行的产品，第一行表格标题除外。为什么标题行被跳过了呢？有两个理由。首先，对象不能把自己作为兄弟标签。任何时候你获取一个标签的兄弟标签，都不会包含这个标签本身。其次，这个函数只调用后面的兄弟标签。例如，如果我们选择一组标签中位于中间位置的一个标签，然后用next_siblings()函数，那么它就只会返回在它后面的兄弟标签。因此，选择标签行然后调用next_siblings，可以选择表格中除了标题行以外的所有行。

### 让标签的选择更具体

如果我们选择bs0bj.table.tr或直接就用bs0bj.tr来获取表格中的第一行，上面的代码也可以获得正确的结果。但是，我们还是采用更长的形式写了一行代码，这可以避免各种意外：

    bs0bj.find("table",{"id":"giftList"}).tr

即使页面上只有一个表格（或其他目标标签），只用标签也很容易丢失细节。另外，页面布局总是不断变化的。一个标签这次是在表格中第一行的位置，没准儿哪天就在第二行或第三行了。如果想让你的爬虫更稳定，最好还是让标签的选择更加具体。如果有属性，就利用标签的属性。

和next_siblings一样，如果你很容易找到一组兄弟标签中的最后一个标签，那么previous_siblings函数也会很有用。当然，还有next_sibling和previous_sibling函数，与next_siblings和previous_siblings的作用类似，只是它们返回的是单个标签，而不是一组标签。

## 5.3 - 父标签的处理

在抓取网页的时候，查找父标签的需求比查找子标签和兄弟标签要少很多。通常情况下，如果以抓取网页内容为目的来观察HTML页面，我们都是从最上层标签开始的，然后思考如何定位我们想要的数据块所在的位置。但是，偶尔在特殊情况下你也会用到BeautifulSoup的父标签查找函数，parent和parents。例如：

In [ ]:
bs0bj.find("img", {"src": "../img/gifts/img1.jpg"}).parent.previous_sibling.get_text()

这一步是如何实现的呢？

# 6 - 正则表达式和BeautifulSoup

让我们看几个例子，待抓取的网页是 http://www.pythonscraping.com/pages/page3.html 。注意观察网页上有几个商品图片——它们的源代码形式如下：

\<img src="../img/gifts/img3.jpg"> 

如果我们想抓取所有图片的URL链接，非常直接的做法就是用findAll("img")抓取所有图片，对吗？但是，有个问题。除了那些明显“多余的”图片（比如，LOGO）之外，新式的网站里都有一些隐藏图片，用于网页布局留白和元素对齐的空白图片，以及一些不容易察觉到的图片标签。总之，你不能仅用商品图片来统计网页上所有的图片。

而且网页的布局也可能会变化，或者，因为某些原因，我们不想通过图片在网页中的位置来查找标签。那么当你想抓取随机分布在网站里的某个元素或数据时，就会出现问题。例如，一些网页的最上面可能有一张商品图片，但是在另一些网页上没有。

解决这类问题的办法，就是直接定位那些标签来查找信息。在本例中，我们直接通过商品图片的文件路径来查找：

In [ ]:
import re
images = bs0bj.findAll("img", {"src": re.compile("\.\.\/img\/gifts/img.*\.jpg")})
for image in images:
    print(image)

# 7 - 获取属性

到目前为止，我们已经介绍过如何获取和过滤标签，以及获取标签里的内容。但是，在网络数据采集时你经常不需要查找标签的内容，而是需要查找标签属性。比如标签\<a>指向的URL链接包含在href属性中，或者\<img>标签的图片文件包含在src属性中，这时获取标签属性就变得非常有用了。

对于一个标签对象，可以用下面的代码获取它的全部属性：

myTag.attrs

要注意这行代码返回的是一个Python字典对象，可以获取和操作这些属性。比如要获取图片的资源位置src，可以用下面这行代码：

myImgTag.attrs\["src"\]

In [ ]:
myImgTag.attrs["src"]

# 8 - Lambda表达式

BeautifulSoup允许我们把特定函数类型当作findAll函数的参数。唯一的限制条件是这些函数必须把一个标签作为参数且返回结果是布尔类型。BeautifulSoup用这个函数来评估它遇到的每个标签对象，最后把评估结果为“真”的标签保留，把其他标签剔除。例如，下面的代码就是获取有两个属性的标签：

In [ ]:
soup.findAll(lambda tag: len(tag.attrs) == 2)

这行代码会找出下面的标签：

\<div class="body" id="content">\</div>

\<spans tyle="color:red" class="title">\</span>

# 8 - 超越BeautifulSoup

lxml这个库（http://lxml.de/） 可以用来解析HTML和XML文档，以非常底层的实现而闻名于世，大部分源代码是用C语言写的。虽然学习它需要花一些时间（其实学习曲线越陡峭，表明你可以越快地学会它），但它在处理绝大多数HTML文档时速度都非常快。

HTML parser这是Python自带的解析库（https://docs.python.org/3/library/html.parser.html）。 因为它不用安装（只要装了Python就有），所以可以很方便地使用。

### <font color=#FF2121>小补充 - 表格属性</font>

| 属性 | 含义 |
|:---:| :---:| 
| \<table>...\</table> | 用于定义一个表格开始和结束 |
| \<th>...\</th> | 定义表头单元格。表格中的文字将以粗体显示，在表格中也可以不用此标签，\<th>标签必须放在\<tr>标签内
| \<tr>...\</tr> | 定义一行标签，一组行标签内可以建立多组由\<td>或\<th>标签所定义的单元格 |
| \<td>...\</td> | 定义单元格标签，一组\<td>标签将将建立一个单元格，\<td>标签必须放在\<tr>标签内 |